In [1]:
# !pip install openpipe-art==0.5.0 langchain-core tenacity datasets vllm faiss-cpu chromadb requests lxml numpy transformers torch gql==3.4.1 peft 
# !pip install langchain-core tenacity datasets vllm

In [2]:
import os
from secretsConfig import oaiKey, wandbKey, openRouterKey  # Add openRouterKey

# Required for RULER judge model
os.environ["OPENAI_API_KEY"] = oaiKey

# Required for Weights & Biases
os.environ["WANDB_API_KEY"] = wandbKey

# Required for OpenRouter (Gemini judge)
os.environ["OPENROUTER_API_KEY"] = openRouterKey  # ADD THIS LINE

if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY is required for RULER functionality.")

if not os.environ.get("WANDB_API_KEY"):
    raise ValueError("WANDB_API_KEY is required for W&B.")

if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("OPENROUTER_API_KEY is required for Gemini judge.")

In [3]:
from IBM_Z_Datathon_RAG.semantic_search import FAISSSemanticSearch
from IBM_Z_Datathon_RAG.KeywordSearch import keyword_search
from IBM_Z_Datathon_RAG.ReadDocumentPart import read_document_part

# Wrap tools with error handling that tracks mistakes
class ToolError:
    """Marker for tool execution errors"""
    def __init__(self, message: str):
        self.message = message

_original_keyword_search = keyword_search
_original_read_document_part = read_document_part

def keyword_search(query: str, num: int = 5) -> str:
    """Safe keyword search wrapper"""
    try:
        return _original_keyword_search(query, num)
    except Exception as e:
        error_msg = f"[TOOL ERROR] keyword_search failed: {str(e)}"
        print(error_msg)
        return error_msg

def read_document_part(part_id: str) -> str:
    """Safe read_document_part wrapper - validates part_id format"""
    try:
        # Validate part_id format (should be like "doc:section:pX", not a search query)
        if " " in part_id or len(part_id) > 100:
            error_msg = f"[INVALID PART_ID] '{part_id[:50]}...' is not a valid part_id format. Part IDs should be like 'doc:section:pX'. Use search tools first to find valid part IDs."
            print(f"[ERROR] Model tried to read invalid part_id: {part_id[:50]}...")
            return error_msg
        
        return _original_read_document_part(part_id)
        
    except FileNotFoundError as e:
        error_msg = f"[PART NOT FOUND] Part ID '{part_id}' does not exist. Use search_keyword or search_semantic first to find valid part IDs."
        print(f"[ERROR] {error_msg}")
        return error_msg
        
    except Exception as e:
        error_msg = f"[TOOL ERROR] Failed to read document part: {str(e)}"
        print(f"[ERROR] {error_msg}")
        return error_msg

print("✅ Tools wrapped with error handling and validation")

✅ Tools wrapped with error handling and validation


In [4]:
from dotenv import load_dotenv
import random

import art
from art.serverless.backend import ServerlessBackend

load_dotenv()

random.seed(42)

# Declare the model - CHANGED TO QWEN3-14B
model = art.TrainableModel(
    name="legal-agent-001",
    project="legal-rag",
    base_model="Qwen/Qwen2.5-14B-Instruct",  # Changed from Qwen2.5-14B-Instruct
)

# Initialize the server
# Training and inference will run on Weights & Biases servers
backend = ServerlessBackend()

# Register the model with the Serverless Backend (sets up logging, inference, and training)
await model.register(backend)

/opt/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from textwrap import dedent
from pydantic import BaseModel, Field
from openai import AsyncOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool
import art

MAX_TURNS = 4


class FinalAnswer(BaseModel):
    answer: str
    source_ids: list[str]


class LegalScenario(BaseModel):
    id: str
    question: str
    gold_answer: str | None = None
    gold_part_ids: list[str] | None = None


class LegalScenarioStep(BaseModel):
    step: int
    scenario: LegalScenario


async def rollout(model: art.Model, legal_scenario_step: LegalScenarioStep) -> art.Trajectory:
    """Execute one trajectory rollout"""
    scenario = legal_scenario_step.scenario
    
    traj = art.Trajectory(
        reward=0.0,
        messages_and_choices=[],
        metadata={"scenario_id": scenario.id, "step": legal_scenario_step.step},
    )

    # YOUR CUSTOM PROMPT HERE
    system_prompt = dedent(
        f"""
        You are a legal research assistant that can search legal documents to answer questions.

        You have access to the following tools:

        - search_keyword(query: str, num: int) -> str: Search using keyword/BM25 search for exact term matches.
        - search_semantic(query: str, num: int) -> str: Search using semantic/vector search for conceptual similarity.
        - read_document_part(part_id: str) -> str: Read a document part by ID. Part IDs use hierarchical format (e.g., A:B:C). To access parent parts, remove the last segment (e.g., A:B:C → parent is A:B).

        You may call one tool per turn, for up to {MAX_TURNS} turns, before giving your final answer.

        In each turn, you should analyze what information you need and respond with EITHER a tool call OR your final answer.

        For tool calls, use this format:
        <think>
        [your reasoning for what to search for and why]
        </think>
        <tool>
        {{"name": "tool_name", "args": {{"query": "search query"}}}}
        </tool>

        When you have enough information, give your final answer in this format:

        <think>
        [your reasoning for the answer]
        </think>
        <answer>
        [your comprehensive answer citing the evidence you found or "I don't know" if you didn't get enough information]

        <sources>
        <source>doc_id_1</source>
        </sources>
        </answer>
        """
    )

    traj.messages_and_choices = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": scenario.question},
    ]

    # Define tools
    def search_keyword_tool(query: str, num: int = 5) -> str:
        return keyword_search(query, num)

    def search_semantic_tool(query: str, num: int = 5) -> str:
        searcher = FAISSSemanticSearch()
        return searcher.search(query, num)

    def read_document_part_tool(part_id: str) -> str:
        return read_document_part(part_id)

    def return_final_answer(answer: str, source_ids: list[str]) -> FinalAnswer:
        return FinalAnswer(answer=answer, source_ids=source_ids)

    tools = [search_keyword_tool, search_semantic_tool, read_document_part_tool, return_final_answer]
    tools_by_name = {t.__name__: t for t in tools}
    traj.tools = [convert_to_openai_tool(t) for t in tools]

    client = AsyncOpenAI(
        base_url=model.inference_base_url,
        api_key=model.inference_api_key,
    )

    for _ in range(MAX_TURNS):
        response = await client.chat.completions.create(
            model=model.get_inference_name(),
            temperature=1,
            messages=traj.messages(),
            tools=traj.tools,
        )

        response_message = response.choices[0].message
        traj.messages_and_choices.append(response.choices[0])

        if not response_message.tool_calls:
            return traj

        try:
            for tool_call in response_message.tool_calls:
                tool_name = tool_call.function.name
                if tool_name in tools_by_name:
                    tool_args = json.loads(tool_call.function.arguments)
                    result = tools_by_name[tool_name](**tool_args)
                    traj.messages_and_choices.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": tool_name,
                        "content": str(result),
                    })

                    if tool_name == "return_final_answer":
                        return traj
        except Exception as e:
            print(f"Error: {e}")
            return traj

    return traj


print("✅ Rollout function defined!")

✅ Rollout function defined!


In [6]:
import json
import os
from litellm import acompletion

# Load your training data
DATA_FILE = "./snippet_data.json"

print(f"Loading data from {DATA_FILE}...")
with open(DATA_FILE, 'r') as f:
    data = json.load(f)

# Convert to LegalScenario objects
training_scenarios = []
for item in data.get("items", []):
    for row in item.get("rows", []):
        sources = row.get("sources", [])
        gold_part_ids = sources if sources else []
        
        training_scenarios.append(
            LegalScenario(
                id=str(row["row_index"]),
                question=row["question"],
                gold_answer=row.get("model_answer", ""),
                gold_part_ids=gold_part_ids
            )
        )

print(f"✅ Loaded {len(training_scenarios)} scenarios")


# Custom RULER function using OpenRouter
async def gemini_ruler_score_group(group: art.TrajectoryGroup) -> art.TrajectoryGroup:
    """Score trajectories using Gemini 2.5 Flash via OpenRouter"""
    
    trajectories = group.trajectories
    if len(trajectories) <= 1:
        for traj in trajectories:
            traj.reward = 0.0
        return group
    
    # Check for tool errors and assign harsh penalties immediately
    for traj in trajectories:
        messages = traj.messages()
        full_conversation = "\n".join([str(m.get("content", "")) for m in messages])
        
        # Harsh penalty for invalid tool usage
        if "[INVALID PART_ID]" in full_conversation or "[TOOL ERROR]" in full_conversation:
            traj.reward = -2.0  # Format/usage error band
            print(f"  ⚠️  Trajectory penalized: -2.0 (invalid tool usage)")
            continue
        
        # Also check if no answer was provided
        if not any("<answer>" in str(m.get("content", "")) for m in messages):
            traj.reward = -1.0  # Wrong answer band
            print(f"  ⚠️  Trajectory penalized: -1.0 (no answer provided)")
            continue
    
    # For trajectories without errors, use Gemini to compare
    valid_trajectories = [t for t in trajectories if t.reward == 0.0]
    
    if len(valid_trajectories) <= 1:
        return group
    
    # Extract responses from valid trajectories
    responses = []
    for traj in valid_trajectories:
        messages = traj.messages()
        if messages:
            last_msg = messages[-1].get("content", "")
            responses.append(last_msg[:500])
        else:
            responses.append("")
    
    # Build comparison prompt
    comparison_text = "\n\n".join([
        f"**Response {i+1}:**\n{resp}"
        for i, resp in enumerate(responses)
    ])
    
    judge_prompt = f"""Compare these {len(responses)} legal research responses and rank them.

Criteria:
1. Correctness and accuracy (most important)
2. Proper citation of sources with part_ids
3. Completeness of answer

Responses:
{comparison_text}

Return ONLY a JSON array of scores from 0.0 to 2.0, one score per response in order.
Higher scores = better responses.
Example: [2.0, 0.5, 1.5]

Your scores:"""
    
    try:
        # Call Gemini via OpenRouter
        response = await acompletion(
            model="openrouter/google/gemini-2.5-flash",
            messages=[{"role": "user", "content": judge_prompt}],
            api_key=os.environ["OPENROUTER_API_KEY"],
            max_tokens=100,
        )
        
        # Parse scores
        result_text = response.choices[0].message.content.strip()
        
        # Extract JSON array
        import re
        json_match = re.search(r'\[[\d\.,\s]+\]', result_text)
        if json_match:
            scores = json.loads(json_match.group())
        else:
            scores = json.loads(result_text)
        
        # Assign scores to valid trajectories
        for traj, score in zip(valid_trajectories, scores):
            traj.reward = float(score)
        
        print(f"  Scores: {scores}")
        
    except Exception as e:
        print(f"  Error in judge: {e}")
        # Fallback: random variation for valid trajectories only
        import random
        for traj in valid_trajectories:
            traj.reward = random.uniform(0.5, 1.5)
    
    return group


# Test the judge
print("\n🧪 Testing Gemini judge via OpenRouter...")

test_scenario = training_scenarios[0]
base_messages = [
    {"role": "system", "content": "You are a legal research agent."},
    {"role": "user", "content": test_scenario.question},
]

good_traj = art.Trajectory(
    messages_and_choices=[
        *base_messages,
        {"role": "assistant", "content": test_scenario.gold_answer},
    ],
    reward=0,
)

bad_traj = art.Trajectory(
    messages_and_choices=[
        *base_messages,
        {"role": "assistant", "content": "I don't know anything about this legal question."},
    ],
    reward=0,
)

test_group = art.TrajectoryGroup(trajectories=[good_traj, bad_traj])

# Score using custom function
judged_group = await gemini_ruler_score_group(test_group)

# Display results
sorted_trajs = sorted(judged_group.trajectories, key=lambda t: t.reward, reverse=True)
for rank, traj in enumerate(sorted_trajs, 1):
    msgs = traj.messages()
    print(f"\nRank {rank}: Score {traj.reward:.3f}")
    print(f"  Response: {msgs[-1]['content'][:80]}...")

print("\n✅ Gemini judge working!")

Loading data from ./snippet_data.json...
✅ Loaded 100 scenarios

🧪 Testing Gemini judge via OpenRouter...
  ⚠️  Trajectory penalized: -1.0 (no answer provided)
  ⚠️  Trajectory penalized: -1.0 (no answer provided)

Rank 1: Score -1.000
  Response: The Marshall Court reasoned that a land grant from a state constitutes a binding...

Rank 2: Score -1.000
  Response: I don't know anything about this legal question....

✅ Gemini judge working!


In [ ]:
from art.utils import iterate_dataset
import wandb
from datetime import datetime

# Initialize W&B with auto-generated run name
wandb.login(key=os.environ["WANDB_API_KEY"])
run = wandb.init(
    project="IBM-Datathon-Z-2025",
    config={
        "model": "Qwen/Qwen2.5-14B-Instruct",
        "groups_per_step": 2,
        "num_epochs": 3,
        "rollouts_per_group": 6,
        "learning_rate": 1e-5,
        "max_steps": 50,
        "max_turns": MAX_TURNS,
    },
    # Remove the name parameter to get auto-generated names with numbers
    # name="legal-rag-rl-training"  # REMOVED THIS LINE
)

# Training config
training_config = run.config

# Create training iterator starting from step 0 (fresh run)
training_iterator = iterate_dataset(
    training_scenarios,
    groups_per_step=training_config["groups_per_step"],
    num_epochs=training_config["num_epochs"],
    initial_step=0,  # CHANGED: Always start from 0 for fresh runs
)

print("🚀 Starting training loop...\n")
print(f"💻 Model inference running on: W&B Serverless (not your A100s)")
print(f"📊 W&B Dashboard: {run.url}")
print(f"🏷️  Run name: {run.name}\n")

step_count = 0

for batch in training_iterator:
    print(f"=== Step {batch.step} | Epoch {batch.epoch} | Epoch Step {batch.epoch_step} ===")
    print(f"Batch: {len(batch.items)} scenarios")
    
    # Create trajectory groups
    groups = []
    for scenario in batch.items:
        groups.append(
            art.TrajectoryGroup(
                (
                    rollout(model, LegalScenarioStep(step=batch.step, scenario=scenario))
                    for _ in range(training_config["rollouts_per_group"])
                )
            )
        )
    
    # Gather trajectories
    finished_groups = await art.gather_trajectory_groups(
        groups,
        pbar_desc="Gathering trajectories",
        max_exceptions=training_config["rollouts_per_group"] * len(batch.items),
    )
    
    # Judge with custom Gemini function
    judged_groups = []
    for group in finished_groups:
        judged_group = await gemini_ruler_score_group(group)
        judged_groups.append(judged_group)
    
    # Calculate metrics before training
    all_rewards = [t.reward for g in judged_groups for t in g.trajectories]
    avg_reward = sum(all_rewards) / len(all_rewards)
    max_reward = max(all_rewards)
    min_reward = min(all_rewards)
    
    # Count trajectories by reward band
    correct_count = sum(1 for r in all_rewards if r >= 1.0)
    idk_count = sum(1 for r in all_rewards if 0.0 <= r < 1.0)
    wrong_count = sum(1 for r in all_rewards if -1.0 <= r < 0.0)
    format_error_count = sum(1 for r in all_rewards if r < -1.0)
    
    # Log to W&B
    wandb.log({
        "step": step_count,  # Use step_count instead of batch.step
        "epoch": batch.epoch,
        "avg_reward": avg_reward,
        "max_reward": max_reward,
        "min_reward": min_reward,
        "correct_count": correct_count,
        "idk_count": idk_count,
        "wrong_count": wrong_count,
        "format_error_count": format_error_count,
        "total_trajectories": len(all_rewards),
    })
    
    print(f"  Rewards: avg={avg_reward:.3f}, max={max_reward:.3f}, min={min_reward:.3f}")
    print(f"  Correct: {correct_count}, IDK: {idk_count}, Wrong: {wrong_count}, Format Errors: {format_error_count}")
    
    # Train on judged trajectories
    await model.delete_checkpoints()
    await model.train(
        judged_groups,
        config=art.TrainConfig(learning_rate=training_config["learning_rate"]),
    )
    
    print(f"✅ Step {step_count} complete\n")
    
    step_count += 1
    
    # Stop after max_steps
    if step_count >= training_config["max_steps"]:
        break

run.finish()
print("🎉 Training complete!")
print(f"📊 View results: {run.url}")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shng2025 (Imperial-College-London-SPQR) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [litellm, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Weave is installed but not imported. Add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


🚀 Starting training loop...

💻 Model inference running on: W&B Serverless (not your A100s)
📊 W&B Dashboard: https://wandb.ai/Imperial-College-London-SPQR/IBM-Datathon-Z-2025/runs/pmt5yxjw
🏷️  Run name: dashing-firebrand-6



Iterating dataset:   0%|          | 0/150 [00:00<?, ?batch/s]

=== Step 0 | Epoch 0 | Epoch Step 0 ===
Batch: 2 scenarios


...

  Scores: [1.5, 1.0, 0.5, 0.5, 0.5, 1.0]
  Scores: [1.7, 1.8, 1.9, 2.0]
  Rewards: avg=1.240, max=2.000, min=0.500
  Correct: 7, IDK: 3, Wrong: 0, Format Errors: 0


Iterating dataset:  16%|█▌        | 24/150 [10:10<53:15, 25.36s/batch]

✅ Step 23 complete

=== Step 24 | Epoch 0 | Epoch Step 24 ===
Batch: 2 scenarios


Gathering trajectories:  83%|████████▎ | 10/12 [00:03<00:00,  2.66it/s, exceptions=2, reward=0, completion_tokens=93.3]


  Scores: [1.5, 1.8, 1.8, 1.9, 1.9]
  Scores: [1.8, 0.5, 1.5, 1.5, 1.8]
  Rewards: avg=1.600, max=1.900, min=0.500
  Correct: 9, IDK: 1, Wrong: 0, Format Errors: 0
